## Testing the server

In [1]:
import requests

SERVER_URL = "http://localhost:8000"

def get_root():
    response = requests.get(f"{SERVER_URL}/")
    return response.json()

## Testing the Inference

In [2]:
import pandas as pd

df = pd.read_csv("../Data/stroke_data_cleaned.csv")  # update with your path

# Show 5 random samples
df.sample(5)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
532,31564,Female,-0.806115,0,0,Yes,Private,Rural,-0.311525,-1.098658,Unknown,0
1439,14709,Male,0.034205,0,0,Yes,Private,Urban,-0.049868,0.615007,never smoked,0
965,5684,Male,-0.142705,0,0,No,Private,Urban,-0.383187,0.024107,formerly smoked,0
4138,2692,Female,1.626390,0,0,Yes,Self-employed,Urban,-0.816772,0.699286,never smoked,0
4645,25783,Female,-1.890570,0,0,No,children,Rural,-0.208850,-1.955490,Unknown,0


In [3]:
def predict_stroke_df(df: pd.DataFrame):
    """
    Takes a DataFrame of stroke patient data (raw features),
    sends each row to the server, and returns predictions in a new DataFrame.

    Expected columns in df:
        age, hypertension, heart_disease, ever_married,
        work_type, residence_type, avg_glucose_level, bmi, smoking_status
    """
    results = []

    for _, row in df.iterrows():
        # Convert row to dictionary (JSON payload)
        patient = {
            "gender": str(row["gender"]),
            "age": float(row["age"]),
            "hypertension": int(row["hypertension"]),
            "heart_disease": int(row["heart_disease"]),
            "ever_married": str(row["ever_married"]),
            "work_type": str(row["work_type"]),
            "Residence_type": str(row["Residence_type"]),
            "avg_glucose_level": float(row["avg_glucose_level"]),
            "bmi": float(row["bmi"]),
            "smoking_status": str(row["smoking_status"])
        }

        # Send to server
        response = requests.post(f"{SERVER_URL}/predict/", json=patient)
        if response.status_code == 200:
            prediction = response.json()
            results.append({**patient, **prediction})
        else:
            results.append({**patient, "stroke_prediction": None, "stroke_probability": None})

    return pd.DataFrame(results)


In [9]:
# Suppose df_sample is your raw stroke dataset
sample_df = df.sample(5).reset_index(drop=True)

# Get predictions
predictions_df = predict_stroke_df(sample_df)

# Show results
predictions_df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke_prediction,stroke_probability
0,Female,-0.010023,0,0,Yes,Private,Urban,-0.786963,-0.255872,never smoked,0,0.300232
1,Male,0.874525,0,0,Yes,Govt_job,Rural,2.058363,-0.003036,formerly smoked,1,0.650409
2,Female,0.299569,0,0,Yes,Govt_job,Urban,-0.560836,0.277893,never smoked,0,0.396235
3,Male,1.537935,0,1,Yes,Private,Urban,2.058363,0.024107,Unknown,1,0.843566
4,Female,-0.275387,0,0,Yes,Private,Rural,1.865734,1.499933,never smoked,0,0.292467
